# Mejoras

* mejorar el codigo y adaptarlo a la api2 de keras
* mejorar la lectura/escritura de los ficheros
* mejora de la organizacion de los ficheros de los pesos

In [1]:
# img_labels & img_list - completed path to file
PATH_TO_LABELS_FILE = "../data/output_dataset/img_labels"
PATH_TO_PATHS_FILE = "../data/output_dataset/img_list"

# output convnet files path
PATH_TO_OUTPUT = "../data/output_convnet"

img_labels = []
img_list = []

with open(PATH_TO_LABELS_FILE,'r') as f_img_labels:
    for line in f_img_labels:
        img_labels.append(line[:-1]) 

with open(PATH_TO_PATHS_FILE,'r') as f_img_list:
    for line in f_img_list:
        img_list.append("../data/"+line[:-1]) # para eliminar el retorno de carro

# fc1 layer with activation relU

In [3]:
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, VGG16
import numpy as np

Using TensorFlow backend.


In [ ]:
# base_model = VGG16(weights='imagenet', include_top=True)

# if last command does not work properly
keras_weights = "keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5"
base_model = VGG16(include_top=True)
model.load_weights(weights_path)

# check for trainable layers
# for i, layer in enumerate(base_model.layers):
#     if layer.trainable:
#         print("layer {0:s} is trainable".format(layer.name))
#     else:
#         print("layer {0:s} is freezed".format(layer.name))

# model architecture
for i, layer in enumerate(base_model.layers):
    print (i, layer.name, layer.output_shape)

In [ ]:
import time
from sys import getsizeof

deep_features = 4096
VGG16_dfmap = np.empty((len(img_list),deep_features))

# init time
ti_dfmap_relU = time.time()

for index, img_path in enumerate(img_list):
    
    index += 1
    
    if not index % 500:
        print("image {0:d} processed".format(index))

    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)

    # img preprocessing
    img = image.load_img(img_path, target_size=(224, 224), interpolation='lanczos')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # model predict
    x = model.predict(x)
    
    VGG16_dfmap[index,:] = np.squeeze(x, axis=0)

# stop time
tf_dfmap_relU = time.time()    
tt_dfmap_relU = tf_dfmap_relU - ti_dfmap_relU
print()
print(time.strftime("%H:%M:%S", time.gmtime(tt_dfmap_relU)))
print()
print("Tamaño en memoria de la matriz de características profundas: {0:.2f}Mb".format(getsizeof(VGG16_dfmap)/float(1<<20)))

In [ ]:
# checking the relU function
VGG16_dfmap

In [ ]:
import h5py
import os

# init time
ti_h5file = time.time()

hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_relU.h5'), 'w')
hf.create_dataset('dfmap', data=VGG16_dfmap) #163.8 mb no compression at all
# hf.create_dataset('dfmap', data=VGG16_dfmap,compression="gzip", compression_opts=9) # 32.4 mb
hf.close()

# stop time
tf_h5file = time.time()
tt_h5file = tf_h5file - ti_h5file
print(time.strftime("%H:%M:%S", time.gmtime(tt_h5file)))

# import pickle
# pickle.dump(VGG16_dfmap, open('../data/output_convnet/VGG16_dfmap_relU_pickle', 'wb'))

# print(os.path.getsize("../data/output_convnet/VGG16_dfmap_relU_pickle")/float(1<<20)) #156 mb

In [ ]:
# file information
import h5py

hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_relU.h5'), 'r')
print(hf.keys())
n1 = hf.get('dfmap')
print(type(n1))
print(n1)

n1 = np.array(n1)
print(type(n1))
print(n1.shape)

hf.close()

# fc1 layer with no activation function

In [ ]:
from keras import backend as K
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.models import Sequential

In [ ]:
# https://github.com/keras-team/keras/blob/master/keras/applications/vgg16.py # estructura implementada en keras
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())

# top layer of the VGG net
model.add(Dense(4096, activation=None,name="fc1")) # activation function set to none
model.add(Dropout(0.5))
model.add(Dense(4096, activation=None,name="fc2")) # activation function set to none
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

model.summary()

In [ ]:
model.load_weights("keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5")
learning_rate = 0.01
model.compile(optimizer=SGD(lr=learning_rate), loss='categorical_crossentropy')

# check for trainable layers
# for i, layer in enumerate(model.layers):
#     if layer.trainable:
#         print("layer {0:s} is trainable".format(layer.name))
#     else:
#         print("layer {0:s} is freezed".format(layer.name))
        
# for i, layer in enumerate(model.layers):
#     print (i, layer.name, layer.output_shape)

In [ ]:
VGG16_dfmap_no_relU = np.empty((len(img_list),deep_features))

# init time
ti_dfmap_no_relU = time.time()

for index, img_path in enumerate(img_list):
    
    if not index % 500:
        print("image {0:d} processed".format(index))

    model_no_relU = Model(input=model.input, output=model.get_layer('fc1').output)

    # pre procesado de imagen
    img = image.load_img(img_path, target_size=(224, 224), interpolation='lanczos')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # model predict
    x = model_no_relU.predict(x)
    
    VGG16_dfmap_no_relU[index,:] = np.squeeze(x, axis=0)

# stop time
tf_dfmap_no_relU = time.time()    
tt_dfmap_no_relU = tf_dfmap_no_relU - ti_dfmap_no_relU
print()
print(time.strftime("%H:%M:%S", time.gmtime(tt_dfmap_no_relU)))
print()
print("Tamaño en memoria de la matriz de características profundas: {0:.2f}Mb".format(getsizeof(VGG16_dfmap_no_relU)/float(1<<20)))

In [ ]:
# checking the activation function set to none
VGG16_dfmap_no_relU 

In [ ]:
# init time
ti_h5file_no_relU = time.time()

hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_no_relU.h5'), 'w')
hf.create_dataset('dfmap', data=VGG16_dfmap) #163.8 mb
# hf.create_dataset('dfmap', data=VGG16_dfmap_no_relU,compression="gzip", compression_opts=9) # 32.4 mb
hf.close()

# stop time
tf_h5file_no_relU = time.time()
tt_h5file_no_relU = tf_h5file_no_relU - ti_h5file_no_relU
print(time.strftime("%H:%M:%S", time.gmtime(tt_h5file_no_relU)))

# import pickle
# pickle.dump(VGG16_dfmap_no_relU, open('../data/output_convnet/VGG16_dfmap_no_relU_pickle', 'wb'))

# print(os.path.getsize("../data/output_convnet/VGG16_dfmap_no_relU_pickle")/float(1<<20)) #156 mb

In [ ]:
hf = h5py.File(os.path.join(PATH_TO_OUTPUT, 'VGG16_dfmap_no_relU.h5'), 'r')
print(hf.keys())
n1 = hf.get('dfmap')
print(type(n1))
print(n1)

n1 = np.array(n1)
print(type(n1))
print(n1.shape)

hf.close()